# Video Detection

## Load Dependent Packages

In [1]:
import torch
import numpy as np
import cv2
from time import time

## Design Detection Class

In [ ]:
class ObjectDetection:
    """
    Class Implements YOLO5 Model to Make Inferences on A Local Video Using OpenCV2.
    """
    def __init__(self, url: str, store: str="../data/VIDEOS/output/demo.mp4"):
        """
        Initialize the class with youtube url and output file.
        :param url: must be as URL, on which prediction is made.
        :param store: a valid output file path.
        """
        self._URL = url
        self.model = self.load_model() # load model
        self.classes = self.model.names # classes of all objects
        self.out_file = store
        self.dict = {}
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu' # device configura

    def get_video_from_url(self):
        """
        Creates a new video streaming object to extract video frame by frame to make prediction on.
        :return: opencv2 video capture object, with lowest quality frame available for video.
        """
        return cv2.VideoCapture(self._URL)

    def load_model(self):
        """
        Loads yolo5 model from pytorch hub.
        :return: trained pytorch model.
        """
        model = torch.hub.load('../models/yolov5/', '../models/yolov5s.pt', source='local', pretrained=True)
        return model

    def score_frame(self, frame):
        """
        Takes a single frame as input, and scores the frame using yolo5 model.
        :param frame: input frame in numpy/list/tuple format.
        :return: labels and coordinates of objects detected by model in the frame.
        """
        self.model.to(self.device)
        frame = [frame]
        results = self.model(frame)
        labels, cord = results.xyxyn[0][:, -1].cpu().numpy(), results.xyxyn[0][:, :-1].cpu().numpy()
        return labels, cord

    def class_to_label(self, x):
        """
        For a given label value, return corresponding string label.
        :param x: numeric label
        :return: corresponding string label
        """
        return self.classes[int(x)]

    def plot_boxes(self, results, frame):
        """
        Takes a frame and its results as input, and plots the bounding boxes and label on to the frame.
        :param results: contains labels and coordinates predicted by model on the given frame.
        :param frame: Frame which has been scored.
        :return: frame with bounding boxes and labels ploted on it.
        """
        labels, cord = results
        n = len(labels)
        x_shape, y_shape = frame.shape[1], frame.shape[0]
        for i in range(n):
            row = cord[i]
            if row[4] >= 0.2:
                x1, y1, x2, y2 = int(row[0]*x_shape), int(row[1]*y_shape), int(row[2]*x_shape), int(row[3]*y_shape)
                # Note the text color, make different text highlight the different color.
                text = self.class_to_label(labels[i])
                if(self.dict.get(text) is not None):
                    bgr = self.dict[text]
                else:
                    bgr = [int(c) for c in list(np.random.choice(range(256), size=3))]
                    self.dict[text] = bgr
                # bgr = (0, 255, 0)
                cv2.rectangle(frame, (x1, y1), (x2, y2), bgr, 2)
                cv2.putText(frame, text, (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.9, bgr, 2)
        return frame

    def __call__(self) -> None:
        """
        This function is called when class is executed, it runs the loop to read the video frame by frame,
        and write the output into a new file.
        :return: None
        """
        player = self.get_video_from_url()
        assert player.isOpened()
        x_shape = int(player.get(cv2.CAP_PROP_FRAME_WIDTH))
        y_shape = int(player.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fourcc = cv2.VideoWriter_fourcc('M', 'P', '4', 'V')
        out = cv2.VideoWriter(self.out_file, fourcc, 20, (x_shape, y_shape))
        while True:
            start_time = time()
            ret, frame = player.read()
            if ret:
                cv2.namedWindow("frame", 0)
                cv2.resizeWindow("frame", 800, 450)
                cv2.imshow('frame', frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    cv2.destroyWindow('frame')
                    break
                results = self.score_frame(frame)
                frame = self.plot_boxes(results, frame)
                end_time = time()
                fps = 1 / np.round(end_time - start_time, 3)
                # print(f"Frames Per Second : {fps}")
                out.write(frame)
            else:
                cv2.destroyWindow('frame')
                break

## Create A New Object and Execute

In [ ]:
demo = ObjectDetection("../data/VIDEOS/input/demo.mp4")
demo()

RuntimeError: Cannot find callable ../models/yolov5s.pt in hubconf